In [1]:
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
 
dataset_path = 'fer2013/fer2013.csv'
image_size=(48,48)
 
def load_fer2013():
    data = pd.read_csv(dataset_path)
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'),image_size)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).as_matrix()
    return faces, emotions
 
def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x
 
faces, emotions = load_fer2013()
faces = preprocess_input(faces)
xtrain, xtest,ytrain,ytest = train_test_split(faces, emotions,test_size=0.2,shuffle=True)

#print("Size {}".format(len(labels)))
#print(labels.shape)
print(xtrain.shape, xtest.shape)
print(ytrain.shape, ytest.shape)

/home/jogel/caffe_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


(28709, 48, 48, 1) (7178, 48, 48, 1)
(28709, 7) (7178, 7)


In [3]:
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Convolution2D, Dropout, Conv2D
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers import Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import SeparableConv2D
from keras import layers
from keras.regularizers import l2
import pandas as pd
import cv2
import numpy as np
 
# parameters
batch_size = 32
num_epochs = 110
input_shape = (48, 48, 1)
verbose = 1
num_classes = 7
patience = 50
base_path = '/home/jogel/Documents/emoRec/FaceEmotion_ID-master/models/'
l2_regularization=0.01
 
# data generator
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)
 
# model parameters
regularization = l2(l2_regularization)
 
# base
img_input = Input(input_shape)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
 
# module 1
residual = Conv2D(16, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(16, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(16, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 2
residual = Conv2D(32, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(32, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(32, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 3
residual = Conv2D(64, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(64, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(64, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
 
# module 4
residual = Conv2D(128, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
x = Conv2D(num_classes, (3, 3), padding='same')(x)
x = GlobalAveragePooling2D()(x)
output = Activation('softmax',name='predictions')(x)
 
model = Model(img_input, output)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()
 
# callbacks
log_file_path = base_path + '_emotion_training.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)
trained_models_path = base_path + '_mini_XCEPTION'
model_names = trained_models_path + '.{epoch:02d}-{val_acc:.2f}.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,save_best_only=True)
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]
 
model.fit_generator(data_generator.flow(xtrain, ytrain,batch_size),
                        steps_per_epoch=len(xtrain) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=(xtest,ytest))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 46, 46, 8)    72          input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 46, 46, 8)    32          conv2d_8[0][0]                   
__________________________________________________________________________________________________
activation_7 (Activation)       (None, 46, 46, 8)    0           batch_normalization_15[0][0]     
__________________________________________________________________________________________________
conv2d_9 (

Epoch 1/110
898/897 [==============================] - 268s 298ms/step - loss: 1.7886 - acc: 0.3204 - val_loss: 1.6734 - val_acc: 0.3643

Epoch 00001: val_loss improved from inf to 1.67344, saving model to /home/jogel/Documents/emoRec/FaceEmotion_ID-master/models/_mini_XCEPTION.01-0.36.hdf5
Epoch 2/110
898/897 [==============================] - 258s 287ms/step - loss: 1.5387 - acc: 0.4212 - val_loss: 1.8508 - val_acc: 0.3600

Epoch 00002: val_loss did not improve from 1.67344
Epoch 3/110
898/897 [==============================] - 267s 297ms/step - loss: 1.4166 - acc: 0.4646 - val_loss: 1.3733 - val_acc: 0.4694

Epoch 00003: val_loss improved from 1.67344 to 1.37331, saving model to /home/jogel/Documents/emoRec/FaceEmotion_ID-master/models/_mini_XCEPTION.03-0.47.hdf5
Epoch 4/110
898/897 [==============================] - 253s 282ms/step - loss: 1.3415 - acc: 0.4961 - val_loss: 1.3490 - val_acc: 0.4985

Epoch 00004: val_loss improved from 1.37331 to 1.34897, saving model to /home/jogel/D

898/897 [==============================] - 236s 263ms/step - loss: 0.9949 - acc: 0.6276 - val_loss: 1.0483 - val_acc: 0.6116

Epoch 00036: val_loss did not improve from 1.04193
Epoch 37/110
898/897 [==============================] - 236s 263ms/step - loss: 0.9982 - acc: 0.6285 - val_loss: 1.0588 - val_acc: 0.6069

Epoch 00037: val_loss did not improve from 1.04193
Epoch 38/110
898/897 [==============================] - 237s 263ms/step - loss: 0.9898 - acc: 0.6340 - val_loss: 1.1761 - val_acc: 0.5691

Epoch 00038: val_loss did not improve from 1.04193
Epoch 39/110
898/897 [==============================] - 236s 263ms/step - loss: 0.9894 - acc: 0.6302 - val_loss: 1.0779 - val_acc: 0.6095

Epoch 00039: val_loss did not improve from 1.04193
Epoch 40/110
898/897 [==============================] - 236s 263ms/step - loss: 0.9857 - acc: 0.6359 - val_loss: 1.1267 - val_acc: 0.6028

Epoch 00040: val_loss did not improve from 1.04193
Epoch 41/110
898/897 [==============================] - 236s 26

898/897 [==============================] - 246s 274ms/step - loss: 0.9113 - acc: 0.6618 - val_loss: 1.0346 - val_acc: 0.6252

Epoch 00076: val_loss did not improve from 1.00337
Epoch 77/110
898/897 [==============================] - 241s 268ms/step - loss: 0.9092 - acc: 0.6617 - val_loss: 1.0091 - val_acc: 0.6279

Epoch 00077: val_loss did not improve from 1.00337
Epoch 78/110
898/897 [==============================] - 236s 263ms/step - loss: 0.9035 - acc: 0.6651 - val_loss: 1.0428 - val_acc: 0.6174

Epoch 00078: val_loss did not improve from 1.00337
Epoch 79/110
898/897 [==============================] - 236s 263ms/step - loss: 0.9024 - acc: 0.6665 - val_loss: 1.1352 - val_acc: 0.5887

Epoch 00079: val_loss did not improve from 1.00337
Epoch 80/110
898/897 [==============================] - 236s 263ms/step - loss: 0.9029 - acc: 0.6674 - val_loss: 1.0501 - val_acc: 0.6220

Epoch 00080: val_loss did not improve from 1.00337
Epoch 81/110
898/897 [==============================] - 237s 26

In [5]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = num_epochs
plt.plot(np.arange(0, N), model.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), model.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), model.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), model.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy fer2013 dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig("EmoTrainFer2013.png")
plt.show()

TypeError: 'History' object is not subscriptable

In [11]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print("Predicting emotion's names on the test set")
y_pred = model.predict(xtest)
#emotions_ = [0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral]

print("Confusion Matrix")
print(confusion_matrix(ytest, y_pred)) #,labels=range(num_classes)))
print("")
print("Classification Report")
print(classification_report(ytest, y_pred, target_names=emotions_))

Predicting emotion's names on the test set
Confusion Matrix


ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [12]:
scores = model.evaluate(xtrain, ytrain, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 72.60%


In [13]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("EmoClassModel.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("EmoClassWeights.h5")
print("Saved model to disk")

Saved model to disk


In [14]:
# load json and create model
json_file = open('EmoClassModel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("EmoClassWeights.h5")
print("Loaded model from disk")
print('Model Summary:')
print(loaded_model.summary())

Loaded model from disk
Model Summary:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 48, 48, 1)    0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 46, 46, 8)    72          input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 46, 46, 8)    32          conv2d_8[0][0]                   
__________________________________________________________________________________________________
activation_7 (Activation)       (None, 46, 46, 8)    0           batch_normalization_15[0][0]     
_______________________________________________________________________

In [15]:
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(xtest, ytest, verbose=1)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

7178/7178 [==============================] - 16s 2ms/step
acc: 65.05%
